In [1]:
%load_ext autoreload
%autoreload 2
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import os
os.environ['KERAS_BACKEND'] = 'theano'

In [3]:
import tensorflow as tf
import numpy as np
import itertools
import keras
import gym

from tqdm import tqdm
from wrappers.envs import AtariStackFrames
from utils.ae import AEConvKeras

Using Theano backend.
WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release.  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

WARNING (theano.gof.compilelock): Overriding existing lock by dead process '29628' (I am process '995')
Using gpu device 0: GeForce GTX 980 (CNMeM is disabled, cuDNN 5105)


In [4]:
model = keras.models.load_model('pacman_uct.h5')

In [5]:
env = AtariStackFrames(gym.make('MsPacman-v0'))

[2017-02-09 03:42:53,195] Making new env: MsPacman-v0


In [6]:
def collect_data(env=None, n_episodes=10, max_steps=5000):
    states = []
    for i in tqdm(range(n_episodes)):
        state = env.reset()
        states.append(state)
        terminal = False
        step = 0
        while not terminal and step < max_steps:
            action = env.action_space.sample()
            state, reward, terminal, _ = env.step(action)
            states.append(state)
            step += 1
    return np.array(states)

def run_episode(env=None, agent=None, max_steps=5000):
    state = env.reset()
    total_reward = 0.
    terminal = False
    step = 0
    while not terminal and step < max_steps:
        action = agent.predict(np.expand_dims(state, 0)).argmax(axis=1)
        state, reward, terminal, _ = env.step(action)
        total_reward += reward
        step += 1
    return total_reward

In [9]:
for i in xrange(20):
    print run_episode(env, model)

3040.0
1570.0
1340.0
360.0
1460.0
540.0
980.0
330.0
620.0
1660.0
1140.0
1300.0
730.0
700.0
460.0
520.0
2440.0
2740.0
950.0


KeyboardInterrupt: 

In [ ]:
model.predict(states[0:1])

In [ ]:
states[0:1].shape

In [ ]:
[x.shape for x in model.layers[-1].get_weights()]

In [ ]:
model.layers[-1].trainable

In [ ]:
tf.Variable(model.trainable_weights[0], trainable=False)